In [0]:
pip install transformers

In [0]:
from google.colab import drive
drive.mount('/content/gdrive')
!git clone https://github.com/Anthony-Alridge/QualityEstimation
%cd 'QualityEstimation'

Mounted at /content/gdrive
Cloning into 'QualityEstimation'...
remote: Enumerating objects: 113, done.
remote: Counting objects: 100% (113/113), done.
remote: Compressing objects: 100% (86/86), done.
remote: Total 113 (delta 50), reused 88 (delta 27), pack-reused 0
Receiving objects: 100% (113/113), 16.61 MiB | 34.77 MiB/s, done.
Resolving deltas: 100% (50/50), done.
/content/QualityEstimation/QualityEstimation/QualityEstimation/QualityEstimation/QualityEstimation/QualityEstimation/QualityEstimation/QualityEstimation


In [0]:
import numpy as np
import pandas as pd
import torch
import transformers as ppb
from torch.utils import data

In [0]:
f = open('data/en-zh/train.enzh.src') # Open file on read mode
lines_en = f.read().split("\n") # Create a list containing all lines
f.close() # Close file

f = open('data/en-zh/train.enzh.mt') # Open file on read mode
lines_de = f.read().split("\n") # Create a list containing all lines
f.close() # Close file

f = open('data/en-zh/train.enzh.scores') # Open file on read mode
scores = f.read().split("\n") # Create a list containing all lines
f.close() # Close file

In [0]:
tokeniser = ppb.DistilBertTokenizer.from_pretrained('distilbert-base-multilingual-cased')

datas = []
for i,j in zip(lines_en, lines_de):
  if i is '':break
  datas.append(tokeniser.encode(i,text_pair=j,max_length=80,pad_to_max_length=True))
data_tensor = torch.LongTensor(datas)



In [0]:
model = ppb.DistilBertForSequenceClassification.from_pretrained('distilbert-base-multilingual-cased', num_labels = 1)
# model.config.num_labels = 1
model.cuda()

DistilBertForSequenceClassification(
  (distilbert): DistilBertModel(
    (embeddings): Embeddings(
      (word_embeddings): Embedding(119547, 768, padding_idx=0)
      (position_embeddings): Embedding(512, 768)
      (LayerNorm): LayerNorm((768,), eps=1e-12, elementwise_affine=True)
      (dropout): Dropout(p=0.1, inplace=False)
    )
    (transformer): Transformer(
      (layer): ModuleList(
        (0): TransformerBlock(
          (attention): MultiHeadSelfAttention(
            (dropout): Dropout(p=0.1, inplace=False)
            (q_lin): Linear(in_features=768, out_features=768, bias=True)
            (k_lin): Linear(in_features=768, out_features=768, bias=True)
            (v_lin): Linear(in_features=768, out_features=768, bias=True)
            (out_lin): Linear(in_features=768, out_features=768, bias=True)
          )
          (sa_layer_norm): LayerNorm((768,), eps=1e-12, elementwise_affine=True)
          (ffn): FFN(
            (dropout): Dropout(p=0.1, inplace=False)
      

In [0]:
scores_ls = []
for s in scores:
  if s is None or s is '':break
  scores_ls.append(float(s))

labels = torch.FloatTensor(scores_ls)

In [0]:
# print(labels.shape)
# print(data_tensor.shape)


In [0]:
my_dataset = data.TensorDataset(data_tensor,labels) # create your datset
my_dataloader = data.DataLoader(my_dataset,batch_size=32,shuffle=True) 

In [0]:
epochs=2
device=torch.device('cuda')
dtype = torch.long
print_every = 100
optimizer = torch.optim.Adamax(model.parameters())
for e in range(epochs):
        for t, (x, y) in enumerate(my_dataloader):
            model.train()  # put model to training mode
            x = x.to(device=device, dtype=dtype)  # move to device, e.g. GPU
            y = y.to(device=device, dtype=torch.float)
            outputs = model(x, labels=y)
            loss = outputs[0]
            optimizer.zero_grad()

            loss.backward()

            # Update the parameters of the model using the gradients
            optimizer.step()

            if t % print_every == 0:
                print('Epoch: %d, Iteration %d, loss = %.4f' % (e, t, loss.item()))
                #check_accuracy(loader_val, model)
                print()

Epoch: 0, Iteration 0, loss = 0.7736

Epoch: 0, Iteration 100, loss = 0.6667

Epoch: 0, Iteration 200, loss = 0.3933

Epoch: 1, Iteration 0, loss = 0.7249

Epoch: 1, Iteration 100, loss = 0.6989

Epoch: 1, Iteration 200, loss = 0.7267



In [0]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [0]:
path = '/content/drive/My Drive/NLP Chinese Models/distilbert/'
model.save_pretrained(path)  # save
tokeniser.save_pretrained(path)  # save

('/content/drive/My Drive/NLP Chinese Models/distilbert/vocab.txt',
 '/content/drive/My Drive/NLP Chinese Models/distilbert/special_tokens_map.json',
 '/content/drive/My Drive/NLP Chinese Models/distilbert/added_tokens.json')